# Linear codes for SSS-based polynomial masking

### The SSS-based masking is a special case of GCM (Generalized Code-based masking)
### $n=5$ shares, $t=2$, $\ell=8$ bits: (5,2)-SSS (Shamir's Secret Sharing)

Parameters:

- $Z=(X + Y_1\alpha_1+Y_2\alpha_1^2, X + Y_1\alpha_2+Y_2\alpha_2^2, X + Y_1\alpha_3+Y_2\alpha_3^2, X + Y_1\alpha_4+Y_2\alpha_4^2, X + Y_1\alpha_5+Y_2\alpha_5^2,)=X\mathbf{G} + Y\mathbf{H}$ where $X, Y=(Y_1, Y_2)$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively, where $\alpha_i$ for $1\leq i\leq 3$ are three public points in SSS-scheme
- $\mathbf{G} = [1, 1, 1, 1, 1]$ and $\mathbf{H} = \left(\begin{matrix} \alpha_1 & \alpha_2 & \alpha_3 & \alpha_4 & \alpha_5 \\ \alpha_1^2 & \alpha_2^2 & \alpha_3^2 & \alpha_4^2 & \alpha_5^2 \end{matrix}\right)$ are two generator matrices of codes $\mathcal{C}$ and $\mathcal{D}$, resp.
- $\alpha_i\in \mathbb{F}_{2^\ell}\backslash\{0\}$ and $\alpha_1\neq\alpha_2\neq\alpha_3\neq\alpha_4\neq\alpha_5$, thus there are ${255}\choose{5}$=8637487551 linear codes for (5,2)-SSS
- Each nonzero element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 254\}$, the corresponding irreducible polynomial is $g(\alpha) = \alpha^8 + \alpha^4 + \alpha^3 + \alpha^2 +1$
- Due to equivalence of linear codes, we simplify the enumeration by choosing $(\alpha_1, \alpha_2, \alpha_3, \alpha_4, \alpha_5)=(\alpha^i, \alpha^j, \alpha^k, \alpha^l,\alpha^r)$ where $i=0$ and $0<j<k<l<r$. Therefore, we get 169362501 linear codes where we still have too many codes
- We further simplify the enumeration by choosing part of linear codes by fixing $j=8$ and then $0<k<l<r$ and $k,l,r\neq 8$. Therefore, we get 2667126 codes. Note that $j=8$ is the best candidate for 2nd share, we follow a greedy solution!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_log(file_name):
    
    pow_ind = []
    dim_all = []
    d_orig_w = []
    d_dual_w = []
    d_dual_b = []
    with open(file_name, 'r') as fp:
        wd = fp.read().split("\n")[:]
        len_all = 0
        for i in range(len(wd)):
            if wd[i].startswith("j, k, l, r ="):
                pow_ind.append([int(i) for i in re.findall(r"\d+", wd[i])])
                len_all = len_all + 1
            elif wd[i].startswith("Dimension:"):
                dim_all.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD orig D (word):"):
                d_orig_w.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD dual D (word):"):
                d_dual_w.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD dual D  (bit):"):
                d_dual_b.append([int(i) for i in re.findall(r"\d+", wd[i]+wd[i+1])])
            else:
                continue
    
    return pow_ind, d_dual_b

## 1. Loading all weight enumerators

In [3]:
pow_ind, d_dual_b = read_log("./magma_paper/gen_codes_sss_5_2_8b.log") # indices and Weight distributions

print(len(pow_ind)) # 2667126 codes in our enumeration
#print(len(d_dual_b))

2667126


### 1.1 Generating values

In [4]:
alpha_all = np.array(['$\\alpha^{%d}$'%i for i in np.arange(255)])
d_all = np.zeros(len(pow_ind))
B_all = np.zeros(len(pow_ind))
alpha_2 = np.zeros(len(pow_ind), dtype=int)
alpha_3 = np.zeros(len(pow_ind), dtype=int)
alpha_4 = np.zeros(len(pow_ind), dtype=int)
alpha_5 = np.zeros(len(pow_ind), dtype=int)
for i in range(len(pow_ind)):
    d_all[i] = d_dual_b[i][2]
    B_all[i] = d_dual_b[i][3]
    alpha_2[i] = pow_ind[i][0]
    alpha_3[i] = pow_ind[i][1]
    alpha_4[i] = pow_ind[i][2]
    alpha_5[i] = pow_ind[i][3]

### 1.2 Defining styles of dataframe

In [5]:
# Set properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '60px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

<span style="color:blue">**We do not show the codes because the huge amount of candidates**</span>.

## 2. Optimal linear codes for (5,2)-SSS-based masking

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=6$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [6]:
# Finding the indices of d^\perp=6
d_index = []
d_index_alpha_2 = []
d_index_alpha_3 = []
d_index_alpha_4 = []
d_index_alpha_5 = []
d_dual_b_best = []
d_D_perp = 6

for i in range(len(d_dual_b)):
    if d_dual_b[i][2] == d_D_perp:
        d_index.append(i)
        d_index_alpha_2.append(pow_ind[i][0])
        d_index_alpha_3.append(pow_ind[i][1])
        d_index_alpha_4.append(pow_ind[i][2])
        d_index_alpha_5.append(pow_ind[i][3])
        d_dual_b_best.append(d_dual_b[i])

d_index = np.array(d_index)
d_index_alpha_2 = np.array(d_index_alpha_2)
d_index_alpha_3 = np.array(d_index_alpha_3)
d_index_alpha_4 = np.array(d_index_alpha_4)
d_index_alpha_5 = np.array(d_index_alpha_5)
#d_dual_b_best = np.array(d_dual_b_best)

In [7]:
print(len(d_index))
print(d_index)

14317
[    751     801     811 ... 2666623 2666642 2666793]


In [8]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

<span style="color:blue">**We only show the first 200 codes because of the limit on file size**</span>.

In [9]:
c_max = 200
df_4 = pd.DataFrame({'$\\alpha_2$': np.array(alpha_all)[d_index_alpha_2[:c_max]], '$\\alpha_3$': np.array(alpha_all)[d_index_alpha_3[:c_max]], 
                     '$\\alpha_4$': np.array(alpha_all)[d_index_alpha_4[:c_max]], '$\\alpha_5$': np.array(alpha_all)[d_index_alpha_5[:c_max]],
                     '$d_{\mathcal{D}}^\perp$': d_all[d_index[:c_max]], '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index[:c_max]], 'Weight Enumerators* (the first 10 elements)': np.array(d_dual_b_best)[:c_max].tolist()})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=30, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for (5,2)-SSS-based masking with $d_{\mathcal{D}}^\perp=6$.')
    .set_table_styles(styles))


,$\alpha_2$,$\alpha_3$,$\alpha_4$,$\alpha_5$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators* (the first 10 elements)
82,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{51}$,$\alpha^{101}$,6,36,"[0, 1, 6, 36, 7, 311, 8, 1196, 9, 4190, 10, 13056, 11, 35250, 12, 85034, 13, 183330, 14, 353934]"
185,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{114}$,$\alpha^{210}$,6,40,"[0, 1, 6, 40, 7, 314, 8, 1201, 9, 4221, 10, 13005, 11, 34919, 12, 85008, 13, 184296, 14, 354448]"
161,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{105}$,$\alpha^{158}$,6,40,"[0, 1, 6, 40, 7, 296, 8, 1259, 9, 4214, 10, 12791, 11, 35184, 12, 85176, 13, 183839, 14, 354640]"
118,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{78}$,$\alpha^{101}$,6,42,"[0, 1, 6, 42, 7, 332, 8, 1216, 9, 4115, 10, 12798, 11, 35235, 12, 85696, 13, 183726, 14, 353482]"
60,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{36}$,$\alpha^{211}$,6,43,"[0, 1, 6, 43, 7, 282, 8, 1268, 9, 4206, 10, 12765, 11, 35337, 12, 85414, 13, 183359, 14, 353521]"
10,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{5}$,$\alpha^{217}$,6,43,"[0, 1, 6, 43, 7, 316, 8, 1182, 9, 4213, 10, 13019, 11, 34984, 12, 85073, 13, 184070, 14, 354380]"
21,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{11}$,$\alpha^{80}$,6,44,"[0, 1, 6, 44, 7, 317, 8, 1154, 9, 4178, 10, 13105, 11, 35176, 12, 85114, 13, 183663, 14, 353864]"
59,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{36}$,$\alpha^{150}$,6,44,"[0, 1, 6, 44, 7, 319, 8, 1191, 9, 4151, 10, 12934, 11, 35305, 12, 85298, 13, 183332, 14, 354166]"
9,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{5}$,$\alpha^{171}$,6,44,"[0, 1, 6, 44, 7, 309, 8, 1213, 9, 4202, 10, 12828, 11, 35186, 12, 85448, 13, 183560, 14, 354196]"
114,$\alpha^{8}$,$\alpha^{1}$,$\alpha^{75}$,$\alpha^{147}$,6,44,"[0, 1, 6, 44, 7, 320, 8, 1210, 9, 4086, 10, 13030, 11, 35284, 12, 84804, 13, 184134, 14, 354702]"


### All possible $B_{d_{\mathcal{D}}^\perp}$ for codes with $d_{\mathcal{D}}^\perp=6$

In [10]:
B_6_value, B_6_num = np.unique(B_all[d_index], return_counts="True")
print("All candidates for $B_6$: \n", B_6_value)
print("Number of codes for each $B_6$: \n", B_6_num)
print("Number of all candidates for $B_6$: ", len(B_6_num))

All candidates for $B_6$: 
 [ 30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.
  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.
  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.
  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.
  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.  99.
 100. 101. 103. 104. 105. 106. 108. 109. 115.]
Number of codes for each $B_6$: 
 [  1   3   2   2   6  13  15  22  33  39  52  82  90 145 172 212 244 246
 286 381 391 411 458 422 492 547 577 539 594 558 576 565 497 507 480 446
 450 389 361 353 295 285 246 227 241 176 139 157 143 115  96  71  72  60
  47  37  35  23  25  22  26  16  23  15  13   6   6   8   7   9   2   5
   1   2   3   1   1   1   1]
Number of all candidates for $B_6$:  79


In [11]:
# Set properties for th, td and caption elements in dataframe
td_props_2 = [('font-size', '13px'), ('text-align', 'center'), ('min-width', '200px')]
# Set table styles
styles_2 = [dict(selector="th", props=th_props), dict(selector="td", props=td_props_2), dict(selector="caption", props=cp_props)]

In [12]:
df_B = pd.DataFrame({'$B_{d_{\mathcal{D}}^\perp}$': B_6_value, 'Frequency (Number of codes)': B_6_num })
df_B = df_B.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_B.style
    .apply(highlight, threshold=30, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. III All possible $B_{d_{\mathcal{D}}^\perp}$ and corresponding number of codes.')
    .set_table_styles(styles_2))


,$B_{d_{\mathcal{D}}^\perp}$,Frequency (Number of codes)
0,30,1
1,31,3
2,32,2
3,33,2
4,34,6
5,35,13
6,36,15
7,37,22
8,38,33
9,39,39


### 2.2 Optimal codes for (5,2)-SSS-based masking

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here $\max\{d_{\mathcal{D}}^\perp\} = 6$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here $\min\{B_{d_{\mathcal{D}}^\perp}\} = 30$

Note that we use two complementary metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information) to assess the side-channel resistance of SSS-based masking with different codes.

As a result of Tab. II, we conclude that the optimal codes for (5,2)-SSS based masking are generated by $\mathbf{H} = \left(\begin{matrix} \alpha_1 & \alpha_2 & \alpha_3 & \alpha_4 & \alpha_5 \\ \alpha_1^2 & \alpha_2^2 & \alpha_3^2 & \alpha_4^2 & \alpha_5^2 \end{matrix}\right)$ where there are nine optimal codes with different $(\alpha_1, \alpha_2, \alpha_3, \alpha_4, \alpha_5)$. Note that permutation on five public points does not change the codes due to equivalence.

The generator matrix of one optimal code 1s: 

- $(\alpha_1, \alpha_3, \alpha_3, \alpha_4, \alpha_5) = (1, \alpha^{8}, \alpha^{90}, \alpha^{92}, \alpha^{192})$
$$
\mathbf{H}_{1}=\left( \begin{matrix} 1 & \alpha^{8} & \alpha^{90} & \alpha^{92} & \alpha^{192} \\ 1 & \alpha^{16} & \alpha^{180} & \alpha^{184} &\alpha^{129} \end{matrix} \right) \in \mathbb{F}_{2^8}^{2\times 5} 
= \left(
 \begin{matrix}
    1~0~0~0~0~0~0~0~0~0~0~0~0~0~0~0~1~0~0~1~1~1~1~0~0~0~1~0~0~1~1~0~0~0~1~0~1~0~0~1 \\
    0~1~0~0~0~0~0~0~0~0~0~0~0~0~0~0~0~1~0~0~1~1~1~1~0~0~0~1~0~0~1~1~1~0~1~0~1~1~0~0 \\
    0~0~1~0~0~0~0~0~0~0~0~0~0~0~0~0~1~0~0~1~1~1~1~1~1~0~1~1~0~0~0~1~0~1~0~1~0~1~1~0 \\
    0~0~0~1~0~0~0~0~0~0~0~0~0~0~0~0~1~1~1~1~0~1~1~1~1~1~1~0~0~0~0~0~0~0~1~0~1~0~1~1 \\
    0~0~0~0~1~0~0~0~0~0~0~0~0~0~0~0~1~1~0~0~0~0~1~1~0~1~1~1~0~0~0~0~1~0~1~0~1~1~0~1 \\
    0~0~0~0~0~1~0~0~0~0~0~0~0~0~0~0~1~1~0~1~1~0~0~1~0~0~1~1~1~0~0~0~1~1~1~0~1~1~1~0 \\
    0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~0~1~1~0~1~0~1~0~0~0~0~0~1~1~1~0~0~0~1~1~1~0~1~1~1 \\
    0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~0~1~1~0~1~0~1~0~0~0~0~0~1~1~1~0~1~0~0~0~0~0~1~1 \\
    0~0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~1~1~1~0~0~0~0~1~1~1~0~1~1~0~1~1~0~1~0~1~1~1~1 \\
    0~0~0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~1~1~1~0~0~0~1~1~0~0~1~1~1~0~1~1~1~0~1~1~1~1 \\
    0~0~0~0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~1~1~1~0~0~0~1~1~0~0~1~1~1~1~1~0~0~1~1~1~1 \\
    0~0~0~0~0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~1~1~1~0~1~0~0~0~1~0~1~1~1~1~0~1~1~1~1~1 \\
    0~0~0~0~0~0~0~0~0~0~0~0~1~0~0~0~0~0~0~0~0~1~1~1~1~1~1~1~1~1~0~1~1~1~0~1~0~1~1~1 \\
    0~0~0~0~0~0~0~0~0~0~0~0~0~1~0~0~1~0~1~1~1~0~1~1~1~1~0~0~0~1~1~0~1~1~0~1~0~0~1~1 \\
    0~0~0~0~0~0~0~0~0~0~0~0~0~0~1~0~1~1~1~0~0~1~0~1~0~1~1~0~0~0~1~1~1~1~0~1~0~0~0~1 \\
    0~0~0~0~0~0~0~0~0~0~0~0~0~0~0~1~1~1~0~0~1~0~1~0~1~0~0~0~1~0~0~1~1~1~0~1~0~0~0~0 
  \end{matrix} 
\right) \normalsize\in \mathbb{F}_2^{16\times 40}
$$ 